In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def extract_reviews(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    data = []
    
    pagination = soup.find("div", class_="styles_pagination__6VmQv")
    last_page = pagination.find_all("a")[-2].text
    last_page = pd.to_numeric(last_page)
    
    for i in range(1, last_page + 1):
        page_url = url + "?page=" + str(i)
        page_response = requests.get(page_url)
        page_soup = BeautifulSoup(page_response.content, 'html.parser')
        
        reviews = page_soup.find_all("div", class_="styles_cardWrapper__LcCPA styles_show__HUXRb styles_reviewCard__9HxJJ")
        
        for review in reviews:
            review_data = {}
            
            review_data["Company"] = soup.find("h1", class_="typography_default__hIMlQ typography_appearance-default__AAY17 title_title__i9V__").span.get_text(strip=True)
            review_data["Customer"] = review.find("span", class_="typography_heading-xxs__QKBS8 typography_appearance-default__AAY17").text
            review_data["Number_review"] = review.find("div", class_="styles_consumerExtraDetails__fxS4S").find("span").text
            review_data["Language"] = review.find("div", class_="typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_detailsIcon__Fo_ua")
            review_data["Title"] = review.find("h2", class_="typography_heading-s__f7029 typography_appearance-default__AAY17").text
            review_data["Date_review"] = review.find("div", class_="styles_reviewHeader__iU9Px").time.text
            reply = review.find("div").find("p", class_="typography_body-m__xgxZ_ typography_appearance-default__AAY17 styles_message__shHhX")
            review_data['Reply'] = reply.text if reply else None
            review_data['Date_reply'] = review.find("div", class_="styles_content__Hl2Mi").time.text if reply else None
            review_data["Rating"] = review.find("section", class_="styles_reviewContentwrapper__zH_9M").div["data-service-review-rating"]
            type_element = review.find_next("div", class_="typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_detailsIcon__yqwWi")
            review_data["Status"] = type_element.find("span").text if type_element else None
            review_data["Experience Date"] = review.find("div", class_="styles_reviewContent__0Q2Tg").text
            
            data.append(review_data)
    
    df = pd.DataFrame(data)
    return df


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import dateparser
import re
import os
import time

In [2]:
younited_url = "https://www.trustpilot.com/review/www.younited-credit.com"
df_younited = extract_reviews(younited_url)
#df_younited.to_csv("../data/raw/younited.csv")

In [3]:
cofidis_url = "https://www.trustpilot.com/review/www.cofidis.fr"
df_cofidis = extract_reviews(cofidis_url)
#df_cofidis.to_csv("../data/raw/cofidis.csv")

In [4]:
floa_url = "https://www.trustpilot.com/review/www.floabank.fr"
df_floa = extract_reviews(floa_url)
#df_floa.to_csv("../data/raw/floa.csv")


In [5]:
orange_url = "https://www.trustpilot.com/review/www.orangebank.fr"
df_orange = extract_reviews(orange_url)
#df_orange.to_csv("../data/raw/orange.csv")

In [6]:
bourso_url = "https://www.trustpilot.com/review/boursorama-banque.com"
df_bourso = extract_reviews(bourso_url)
#df_bourso.to_csv("../data/raw/bourso.csv")

In [7]:
anytime_url = "https://www.trustpilot.com/review/anyti.me"
df_anytime = extract_reviews(anytime_url)
#df_anytime.to_csv("../data/raw/anytime.csv")

In [8]:
df_all = pd.concat([df_younited, df_cofidis, df_orange, df_floa, df_bourso, df_anytime], axis=0)

df_all = df_all.reset_index(drop=True)
df_all

,Company,Customer,Number_review,Language,Title,Date_review,Reply,Date_reply,Rating,Status,Experience Date
0,Younited Credit,Alexis,2 reviews,"[[[], []], [FR]]",Very good service,"Oct 23, 2023","Bonjour,Nous sommes enchantés de vous lire et ...","Oct 24, 2023",5,Verified,Very good serviceDate of experience: October 1...
1,Younited Credit,Exo Saru,3 reviews,"[[[], []], [FR]]",Super simple,"Aug 24, 2023","Bonjour,Nous vous remercions d’avoir pris le t...","Aug 25, 2023",5,Verified,"Super simpleSuper simple, très rapide, interfa..."
2,Younited Credit,Getifra,3 reviews,"[[[], []], [FR]]",Younited Credit a démontré son…,"Aug 5, 2023","Bonjour,Nous vous remercions d’avoir pris le t...","Aug 8, 2023",5,Verified,Younited Credit a démontré son…Younited Credit...
3,Younited Credit,Duval,9 reviews,"[[[], []], [FR]]",Rapide efficace simple un pur bonheur…,"Nov 28, 2022","Bonjour,Merci d’avoir pris le temps de nous fa...","Nov 30, 2022",5,Verified,Rapide efficace simple un pur bonheur…Rapide e...
4,Younited Credit,Soso,2 reviews,"[[[], []], [FR]]",Younited une procédure simple,"Feb 25, 2023","Bonjour,Nous vous remercions pour ce commentai...","Feb 27, 2023",5,Verified,Younited une procédure simpleYounited permet u...
...,...,...,...,...,...,...,...,...,...,...,...
2438,Anytime,AMIR TAOUBAS,1 review,"[[[], []], [DE]]",Service parfait,"Mar 3, 2023","Bonjour 😁, Merci pour votre super avis ! A trè...","Mar 6, 2023",5,Invited,"Service parfaitDate of experience: March 02, 2023"
2439,Anytime,HOCINE TABERKOKT,1 review,"[[[], []], [FR]]",Excellente communication,"May 16, 2022","Bonjour 🫶, Merci beaucoup pour l'avis positif ...","May 17, 2022",5,Invited,Excellente communicationDate of experience: Ma...
2440,Anytime,TOTO JOSUE WENCESLAS KOUAME,1 review,"[[[], []], [FR]]",Excellent service client,"Mar 8, 2023","Bonjour Toto Josue, Merci de partager votre av...","Mar 9, 2023",5,Invited,Excellent service clientDate of experience: Ja...
2441,Anytime,SHOLA SANNI,1 review,"[[[], []], [FR]]",Nice job and keep it up,"Apr 11, 2022",Bonjour 😁Merci beaucoup pour cet avis positif....,"Apr 13, 2022",5,Invited,Nice job and keep it upDate of experience: Apr...


In [9]:
df_all["Experience Date"]


0       Very good serviceDate of experience: October 1...
1       Super simpleSuper simple, très rapide, interfa...
2       Younited Credit a démontré son…Younited Credit...
3       Rapide efficace simple un pur bonheur…Rapide e...
4       Younited une procédure simpleYounited permet u...
                              ...                        
2438    Service parfaitDate of experience: March 02, 2023
2439    Excellente communicationDate of experience: Ma...
2440    Excellent service clientDate of experience: Ja...
2441    Nice job and keep it upDate of experience: Apr...
2442    excellentexcellente interventionDate of experi...
Name: Experience Date, Length: 2443, dtype: object

In [10]:
df_all.isnull().sum()

Company               0
Customer              0
Number_review         0
Language              3
Title                 0
Date_review           0
Reply              1008
Date_reply         1008
Rating                0
Status               47
Experience Date       0
dtype: int64

In [11]:
df_all[["Number_review", "review"]]= df_all["Number_review"].str.split(" ", n = 1, expand = True)
df_all[["Number_review", "review"]]

,Number_review,review
0,2,reviews
1,3,reviews
2,3,reviews
3,9,reviews
4,2,reviews
...,...,...
2438,1,review
2439,1,review
2440,1,review
2441,1,review


In [12]:
df_all[["ExperienceDate", "Date_experience"]]= df_all["Experience Date"].str.split(":", n = 1, expand = True)
df_all[["ExperienceDate", "Date_experience"]]

,ExperienceDate,Date_experience
0,Very good serviceDate of experience,"October 18, 2023"
1,"Super simpleSuper simple, très rapide, interfa...","August 17, 2023"
2,Younited Credit a démontré son…Younited Credit...,"July 27, 2023"
3,Rapide efficace simple un pur bonheur…Rapide e...,"November 28, 2022"
4,Younited une procédure simpleYounited permet u...,"February 19, 2023"
...,...,...
2438,Service parfaitDate of experience,"March 02, 2023"
2439,Excellente communicationDate of experience,"May 16, 2022"
2440,Excellent service clientDate of experience,"January 31, 2023"
2441,Nice job and keep it upDate of experience,"April 11, 2022"


In [13]:
df_all[["Experience", "Date"]]= df_all["Experience Date"].str.split("Date", n = 1, expand = True)
df_all['Experience'][2]

'Younited Credit a démontré son…Younited Credit a démontré son professionnalisme dans le montage de mon dossier de financement. Avec mes sincères remerciements.Cordialement votre.'

In [14]:
new_df = df_all[["Company", "Customer", "Number_review", "Language", "Title", "Date_review", "Reply", "Date_reply", "Rating", "Status", "Experience", "Date_experience"]]
new_df

,Company,Customer,Number_review,Language,Title,Date_review,Reply,Date_reply,Rating,Status,Experience,Date_experience
0,Younited Credit,Alexis,2,"[[[], []], [FR]]",Very good service,"Oct 23, 2023","Bonjour,Nous sommes enchantés de vous lire et ...","Oct 24, 2023",5,Verified,Very good service,"October 18, 2023"
1,Younited Credit,Exo Saru,3,"[[[], []], [FR]]",Super simple,"Aug 24, 2023","Bonjour,Nous vous remercions d’avoir pris le t...","Aug 25, 2023",5,Verified,"Super simpleSuper simple, très rapide, interfa...","August 17, 2023"
2,Younited Credit,Getifra,3,"[[[], []], [FR]]",Younited Credit a démontré son…,"Aug 5, 2023","Bonjour,Nous vous remercions d’avoir pris le t...","Aug 8, 2023",5,Verified,Younited Credit a démontré son…Younited Credit...,"July 27, 2023"
3,Younited Credit,Duval,9,"[[[], []], [FR]]",Rapide efficace simple un pur bonheur…,"Nov 28, 2022","Bonjour,Merci d’avoir pris le temps de nous fa...","Nov 30, 2022",5,Verified,Rapide efficace simple un pur bonheur…Rapide e...,"November 28, 2022"
4,Younited Credit,Soso,2,"[[[], []], [FR]]",Younited une procédure simple,"Feb 25, 2023","Bonjour,Nous vous remercions pour ce commentai...","Feb 27, 2023",5,Verified,Younited une procédure simpleYounited permet u...,"February 19, 2023"
...,...,...,...,...,...,...,...,...,...,...,...,...
2438,Anytime,AMIR TAOUBAS,1,"[[[], []], [DE]]",Service parfait,"Mar 3, 2023","Bonjour 😁, Merci pour votre super avis ! A trè...","Mar 6, 2023",5,Invited,Service parfait,"March 02, 2023"
2439,Anytime,HOCINE TABERKOKT,1,"[[[], []], [FR]]",Excellente communication,"May 16, 2022","Bonjour 🫶, Merci beaucoup pour l'avis positif ...","May 17, 2022",5,Invited,Excellente communication,"May 16, 2022"
2440,Anytime,TOTO JOSUE WENCESLAS KOUAME,1,"[[[], []], [FR]]",Excellent service client,"Mar 8, 2023","Bonjour Toto Josue, Merci de partager votre av...","Mar 9, 2023",5,Invited,Excellent service client,"January 31, 2023"
2441,Anytime,SHOLA SANNI,1,"[[[], []], [FR]]",Nice job and keep it up,"Apr 11, 2022",Bonjour 😁Merci beaucoup pour cet avis positif....,"Apr 13, 2022",5,Invited,Nice job and keep it up,"April 11, 2022"


In [15]:
new_df.groupby(by="Status").describe()

Company                              Customer         \
             count unique              top freq    count unique   
Status                                                            
Invited       1397      6      Orange Bank  733     1397   1290   
Merged         433      1       BoursoBank  433      433    426   
Redirected      18      1         Floabank   18       18     17   
Verified       548      1  Younited Credit  548      548    428   

                                  Number_review         ... Rating        \
                         top freq         count unique  ...    top  freq   
Status                                                  ...                
Invited             Philippe    9          1397     20  ...      5  1135   
Merged                Client    5           433     21  ...      5   369   
Redirected  Peter Mozelewski    2            18      8  ...      5    14   
Verified             cliente   53           548     13  ...      5   472   

           Experience                                                  \
                count unique                                 top freq   
Status                                                                  
Invited          1397   1232                    Simple et rapide   22   
Merged            433    381                    simple et rapide   11   
Redirected         18     18  Rapide et efficace via WhatsApp !!    1   
Verified          548    460                    Simple et rapide   22   

           Date_experience                                  
                     count unique                 top freq  
Status                                                      
Invited               1397    614      April 22, 2022   26  
Merged                 433    327   December 01, 2021    5  
Redirected              18     18    October 31, 2023    1  
Verified               548    443     August 08, 2020    7  

[4 rows x 44 columns]

In [16]:
new_df[["Reply", "Date_reply"]].isna().sum()


Reply         1008
Date_reply    1008
dtype: int64

In [17]:
new_df[["Status"]] = new_df[["Status"]].fillna(method = "ffill")
new_df[["Status"]]

c:\Users\yaya-\anaconda3\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Status
0,Verified
1,Verified
2,Verified
3,Verified
4,Verified
...,...
2438,Invited
2439,Invited
2440,Invited
2441,Invited


In [18]:
new_df.isna().sum()

Company               0
Customer              0
Number_review         0
Language              3
Title                 0
Date_review           0
Reply              1008
Date_reply         1008
Rating                0
Status                0
Experience            0
Date_experience       0
dtype: int64

In [19]:
new_df.dtypes

Company            object
Customer           object
Number_review      object
Language           object
Title              object
Date_review        object
Reply              object
Date_reply         object
Rating             object
Status             object
Experience         object
Date_experience    object
dtype: object

In [20]:
new_df[["Number_review", "Rating"]] = new_df[["Number_review", "Rating"]].astype("int64")
new_df.dtypes

c:\Users\yaya-\anaconda3\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


Company            object
Customer           object
Number_review       int64
Language           object
Title              object
Date_review        object
Reply              object
Date_reply         object
Rating              int64
Status             object
Experience         object
Date_experience    object
dtype: object

In [21]:
#! pip install dateparser


In [22]:
#import pandas as pd
import dateparser

# Convertir les colonnes "Date_review", "Date_reply" et "Date_experience" en dates
new_df["Date_reply"] = new_df["Date_reply"].fillna(new_df["Date_review"])
new_df['Date_review'] = new_df['Date_review'].apply(lambda x: dateparser.parse(x).strftime('%Y-%m-%d') if isinstance(x, str) and dateparser.parse(x) is not None else x)
#new_df['Date_experience'] = new_df['Date_experience'].apply(lambda x: dateparser.parse(x).strftime('%Y-%m-%d') if isinstance(x, str) and dateparser.parse(x) is not None else x)
#new_df['Date_reply'] = new_df['Date_reply'].apply(lambda x: dateparser.parse(x).strftime('%Y-%m-%d') if isinstance(x, str) and dateparser.parse(x) is not None else x)
# Vérifier le résultat

#new_df.sort_values(by = "Date_review", ascending= False)




C:\Users\yaya-\AppData\Local\Temp/ipykernel_13848/992646271.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["Date_reply"] = new_df["Date_reply"].fillna(new_df["Date_review"])
C:\Users\yaya-\AppData\Local\Temp/ipykernel_13848/992646271.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Date_review'] = new_df['Date_review'].apply(lambda x: dateparser.parse(x).strftime('%Y-%m-%d') if isinstance(x, str) and dateparser.parse(x) is not None else x)


In [23]:
new_df[new_df['Date_review'].str.contains("Updated")]

,Company,Customer,Number_review,Language,Title,Date_review,Reply,Date_reply,Rating,Status,Experience,Date_experience
71,Younited Credit,DANIEL,3,"[[[], []], [FR]]",SERVICE TOP,"Updated Aug 14, 2023","Bonjour,Nous vous remercions d’avoir pris le t...","Aug 14, 2023",5,Verified,SERVICE TOPefficacité simplicité et RAPIDITE !...,"August 01, 2023"
327,Younited Credit,Georges,3,"[[[], []], [FR]]",Younited,"Updated Feb 11, 2021",None,"Updated Feb 11, 2021",5,Verified,YounitedYounited - rapide simple et efficace !,"February 11, 2021"
335,Younited Credit,Aliana,1,"[[[], []], [FR]]",Younited Crédit...je recommande...dispositif s...,"Updated Dec 22, 2020","Bonjour Madame, Nous vous remercions pour ce c...","Dec 21, 2020",5,Verified,Younited Crédit...je recommande...dispositif s...,"December 19, 2020"
376,Younited Credit,Monsieur ISMAIL,4,"[[[], []], [FR]]",bon service simple et rapide det fiable.,"Updated Aug 8, 2020",None,"Updated Aug 8, 2020",5,Verified,bon service simple et rapide det fiable.bon se...,"August 08, 2020"
421,Younited Credit,Vincent,2,"[[[], []], [FR]]",Très contente de younited c simple…,"Updated Jan 19, 2020",None,"Updated Jan 19, 2020",5,Verified,Très contente de younited c simple…Très conten...,"January 19, 2020"
423,Younited Credit,Dominique,1,"[[[], []], [TH]]",Evaluation rapide.,"Updated Jan 9, 2020",None,"Updated Jan 9, 2020",5,Verified,Evaluation rapide.,"January 09, 2020"
426,Younited Credit,Max,1,"[[[], []], [FR]]",finncement adaptable à nos besoin.,"Updated Dec 4, 2019","Bonjour Monsieur ,Nous vous remercions d’avoir...","Dec 4, 2019",5,Verified,finncement adaptable à nos besoin.financement ...,"December 04, 2019"
438,Younited Credit,PAUL,5,"[[[], []], [FR]]",Very efficient and professional,"Updated Nov 3, 2019",None,"Updated Nov 3, 2019",5,Verified,Very efficient and professionalVery efficient ...,"November 03, 2019"
447,Younited Credit,Nicolas,3,"[[[], []], [FR]]",très content,"Updated Oct 9, 2019",None,"Updated Oct 9, 2019",4,Verified,"très contentsimple, rapide et efficace je reco...","October 09, 2019"
462,Younited Credit,Corinne,2,"[[[], []], [FR]]",rapide et simple,"Updated Aug 7, 2019",None,"Updated Aug 7, 2019",4,Verified,rapide et simple,"August 07, 2019"


In [24]:
import re

# Create a regular expression to match the date format
date_regex = re.compile(r'^\d{4}-\d{2}-\d{2}$')

# Identify the invalid dates
invalid_dates = []
for date in new_df['Date_review']:
    if not date_regex.match(date):
        invalid_dates.append(date)

# Print the number of invalid dates
print(len(invalid_dates))


24


In [25]:
min_date = min(new_df['Date_review'])
new_df['Date_review'] = new_df['Date_review'].apply(lambda x: min_date if not date_regex.match(x) else x)


C:\Users\yaya-\AppData\Local\Temp/ipykernel_13848/3766287997.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Date_review'] = new_df['Date_review'].apply(lambda x: min_date if not date_regex.match(x) else x)


In [26]:
#import datetime

# Replace the invalid dates with the current date
#for date in invalid_dates:
    #new_df['Date_review'].loc[new_df['Date_review'] == date] = datetime.date.today()

# Print the DataFrame
new_df.dtypes


Company            object
Customer           object
Number_review       int64
Language           object
Title              object
Date_review        object
Reply              object
Date_reply         object
Rating              int64
Status             object
Experience         object
Date_experience    object
dtype: object

In [27]:
import pandas as pd
import dateparser

#Convertir la colonne "Date_reply" en dates
new_df["Date_reply"] = new_df["Date_reply"].apply(lambda x: dateparser.parse(x) if isinstance(x, str) else x)
#Remplacer les valeurs manquantes de "Date_reply" par les valeurs de "Date_review"
new_df["Date_reply"] = new_df.apply(lambda row: row["Date_review"] if pd.isna(row["Date_reply"]) else row["Date_reply"], axis=1)
#Convertir la colonne "Date_reply" en dates
new_df["Date_experience"] = new_df["Date_experience"].apply(lambda x: dateparser.parse(x) if isinstance(x, str) else x)
#Remplacer les valeurs manquantes de "Date_reply" par les valeurs de "Date_review"
new_df["Date_experience"] = new_df.apply(lambda row: row["Date_experience"] if pd.isna(row["Date_experience"]) else row["Date_experience"], axis=1)
#Convertir la colonne "Date_reply" en dates
new_df["Date_review"] = new_df["Date_review"].apply(lambda x: dateparser.parse(x) if isinstance(x, str) else x)
#Remplacer les valeurs manquantes de "Date_reply" par les valeurs de "Date_review"
new_df["Date_review"] = new_df.apply(lambda row: row["Date_review"] if pd.isna(row["Date_review"]) else row["Date_review"], axis=1)

# Vérifier le résultat

#new_df.sort_values(by = "Date_review", ascending= False)
new_df.dtypes

C:\Users\yaya-\AppData\Local\Temp/ipykernel_13848/1703669963.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["Date_reply"] = new_df["Date_reply"].apply(lambda x: dateparser.parse(x) if isinstance(x, str) else x)
C:\Users\yaya-\AppData\Local\Temp/ipykernel_13848/1703669963.py:7: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype={value.dtype})
  new_df["Date_reply"] = new_df.apply(lambda row: row["Date_review"] if pd.isna(row["Date_reply"]) else row["Date_reply"], axis=1)
C:\Users\yaya-\AppData\Local\Temp/ipykernel_13848/1703669963.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

Company                    object
Customer                   object
Number_review               int64
Language                   object
Title                      object
Date_review        datetime64[ns]
Reply                      object
Date_reply         datetime64[ns]
Rating                      int64
Status                     object
Experience                 object
Date_experience    datetime64[ns]
dtype: object

In [28]:
# Fonction de conversion personnalisée
def convert_review_date(date_str):
    try:
        parsed_date = dateparser.parse(date_str)
        if parsed_date:
            return parsed_date.strftime('%Y-%m-%d')
    except Exception:
        pass
    return date_str

# Appliquer la fonction de conversion à la colonne Date_review
new_df['Date_review'] = new_df['Date_review'].apply(convert_review_date)
#new_df["Response_time"] = new_df["Date_reply"] - new_df["Date_review"]
new_df['Date_reply'] = new_df['Date_reply'].apply(convert_review_date)

new_df

C:\Users\yaya-\AppData\Local\Temp/ipykernel_13848/21958708.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Date_review'] = new_df['Date_review'].apply(convert_review_date)
C:\Users\yaya-\AppData\Local\Temp/ipykernel_13848/21958708.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Date_reply'] = new_df['Date_reply'].apply(convert_review_date)


,Company,Customer,Number_review,Language,Title,Date_review,Reply,Date_reply,Rating,Status,Experience,Date_experience
0,Younited Credit,Alexis,2,"[[[], []], [FR]]",Very good service,2023-10-23,"Bonjour,Nous sommes enchantés de vous lire et ...",2023-10-24,5,Verified,Very good service,2023-10-18
1,Younited Credit,Exo Saru,3,"[[[], []], [FR]]",Super simple,2023-08-24,"Bonjour,Nous vous remercions d’avoir pris le t...",2023-08-25,5,Verified,"Super simpleSuper simple, très rapide, interfa...",2023-08-17
2,Younited Credit,Getifra,3,"[[[], []], [FR]]",Younited Credit a démontré son…,2023-08-05,"Bonjour,Nous vous remercions d’avoir pris le t...",2023-08-08,5,Verified,Younited Credit a démontré son…Younited Credit...,2023-07-27
3,Younited Credit,Duval,9,"[[[], []], [FR]]",Rapide efficace simple un pur bonheur…,2022-11-28,"Bonjour,Merci d’avoir pris le temps de nous fa...",2022-11-30,5,Verified,Rapide efficace simple un pur bonheur…Rapide e...,2022-11-28
4,Younited Credit,Soso,2,"[[[], []], [FR]]",Younited une procédure simple,2023-02-25,"Bonjour,Nous vous remercions pour ce commentai...",2023-02-27,5,Verified,Younited une procédure simpleYounited permet u...,2023-02-19
...,...,...,...,...,...,...,...,...,...,...,...,...
2438,Anytime,AMIR TAOUBAS,1,"[[[], []], [DE]]",Service parfait,2023-03-03,"Bonjour 😁, Merci pour votre super avis ! A trè...",2023-03-06,5,Invited,Service parfait,2023-03-02
2439,Anytime,HOCINE TABERKOKT,1,"[[[], []], [FR]]",Excellente communication,2022-05-16,"Bonjour 🫶, Merci beaucoup pour l'avis positif ...",2022-05-17,5,Invited,Excellente communication,2022-05-16
2440,Anytime,TOTO JOSUE WENCESLAS KOUAME,1,"[[[], []], [FR]]",Excellent service client,2023-03-08,"Bonjour Toto Josue, Merci de partager votre av...",2023-03-09,5,Invited,Excellent service client,2023-01-31
2441,Anytime,SHOLA SANNI,1,"[[[], []], [FR]]",Nice job and keep it up,2022-04-11,Bonjour 😁Merci beaucoup pour cet avis positif....,2022-04-13,5,Invited,Nice job and keep it up,2022-04-11


In [29]:
#new_df["Date_reply"] = new_df["Date_reply"].fillna(new_df["Date_review"])
new_df["Response_time"] = new_df["Date_reply"] - new_df["Date_review"]
new_df.isnull().sum()

C:\Users\yaya-\AppData\Local\Temp/ipykernel_13848/1002915204.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["Response_time"] = new_df["Date_reply"] - new_df["Date_review"]


Company               0
Customer              0
Number_review         0
Language              3
Title                 0
Date_review           0
Reply              1008
Date_reply            0
Rating                0
Status                0
Experience            0
Date_experience      32
Response_time         0
dtype: int64

In [30]:
new_df["Response_time"]

0      1 days
1      1 days
2      3 days
3      2 days
4      2 days
        ...  
2438   3 days
2439   1 days
2440   1 days
2441   2 days
2442   0 days
Name: Response_time, Length: 2443, dtype: timedelta64[ns]

import pandas as pd
import dateparser

# Convert "Date_review" column to dates
new_df.loc[:, "Date_review"] = new_df["Date_review"].apply(lambda x: dateparser.parse(x) if isinstance(x, str) else x)

# Convert "Date_reply" column to dates
new_df.loc[:, "Date_reply"] = new_df["Date_reply"].apply(lambda x: dateparser.parse(x) if isinstance(x, str) else x)

# Convert "Date_experience" column to dates
new_df.loc[:, "Date_experience"] = new_df["Date_experience"].apply(lambda x: dateparser.parse(x) if isinstance(x, str) else x)


import pandas as pd
import dateparser

Remplacer les valeurs manquantes (NaN) de la colonne "Date_reply" par celles de la colonne "Date_review"
new_df["Date_reply"].fillna(new_df["Date_review"], inplace=True)

Convertir les colonnes "Date_review", "Date_reply" et "Date_experience" en dates
new_df["Date_review"] = new_df["Date_review"].apply(lambda x: dateparser.parse(x) if isinstance(x, str) else x)
new_df["Date_reply"] = new_df["Date_reply"].apply(lambda x: dateparser.parse(x) if isinstance(x, str) else x)
new_df["Date_experience"] = new_df["Date_experience"].apply(lambda x: dateparser.parse(x) if isinstance(x, str) else x)
new_df.dtypes

In [31]:
new_df.isnull().sum()


Company               0
Customer              0
Number_review         0
Language              3
Title                 0
Date_review           0
Reply              1008
Date_reply            0
Rating                0
Status                0
Experience            0
Date_experience      32
Response_time         0
dtype: int64

In [32]:
new_df[["Reply"]] = new_df[["Reply"]].fillna("No Reply")
#new_df[["Date_review", "Date_reply", "Date_experience"]] = new_df[["Date_review", "Date_reply", "Date_experience"]].fillna(0)
new_df.dtypes

c:\Users\yaya-\anaconda3\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


Company                     object
Customer                    object
Number_review                int64
Language                    object
Title                       object
Date_review         datetime64[ns]
Reply                       object
Date_reply          datetime64[ns]
Rating                       int64
Status                      object
Experience                  object
Date_experience     datetime64[ns]
Response_time      timedelta64[ns]
dtype: object

In [33]:
#new_df = new_df.drop("Date_reply", axis = 1)
new_df = new_df.dropna(how="any")
new_df.isna().sum()


Company            0
Customer           0
Number_review      0
Language           0
Title              0
Date_review        0
Reply              0
Date_reply         0
Rating             0
Status             0
Experience         0
Date_experience    0
Response_time      0
dtype: int64

In [34]:
new_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2408 entries, 0 to 2442
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype          
---  ------           --------------  -----          
 0   Company          2408 non-null   object         
 1   Customer         2408 non-null   object         
 2   Number_review    2408 non-null   int64          
 3   Language         2408 non-null   object         
 4   Title            2408 non-null   object         
 5   Date_review      2408 non-null   datetime64[ns] 
 6   Reply            2408 non-null   object         
 7   Date_reply       2408 non-null   datetime64[ns] 
 8   Rating           2408 non-null   int64          
 9   Status           2408 non-null   object         
 10  Experience       2408 non-null   object         
 11  Date_experience  2408 non-null   datetime64[ns] 
 12  Response_time    2408 non-null   timedelta64[ns]
dtypes: datetime64[ns](3), int64(2), object(7), timedelta64[ns](1)
memory usage: 26

In [35]:
columns_to_convert = ['Date_review', 'Date_experience', 'Date_reply']

for col in columns_to_convert:
    new_df[col] = pd.to_datetime(new_df[col]).dt.strftime('%Y-%m-%d')

C:\Users\yaya-\AppData\Local\Temp/ipykernel_13848/1252918539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[col] = pd.to_datetime(new_df[col]).dt.strftime('%Y-%m-%d')


C:\Users\yaya-\AppData\Local\Temp/ipykernel_13848/1252918539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[col] = pd.to_datetime(new_df[col]).dt.strftime('%Y-%m-%d')
C:\Users\yaya-\AppData\Local\Temp/ipykernel_13848/1252918539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[col] = pd.to_datetime(new_df[col]).dt.strftime('%Y-%m-%d')


In [36]:
new_df.dtypes

Company                     object
Customer                    object
Number_review                int64
Language                    object
Title                       object
Date_review                 object
Reply                       object
Date_reply                  object
Rating                       int64
Status                      object
Experience                  object
Date_experience             object
Response_time      timedelta64[ns]
dtype: object

In [37]:
new_df["Response_time"] = new_df["Response_time"].astype(str)
new_df[["Response_time", "Day"]] = new_df["Response_time"].str.split(" ", n=1, expand=True)
new_df = new_df.drop("Day", axis=1)
new_df

C:\Users\yaya-\AppData\Local\Temp/ipykernel_13848/215425970.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["Response_time"] = new_df["Response_time"].astype(str)
c:\Users\yaya-\anaconda3\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Company,Customer,Number_review,Language,Title,Date_review,Reply,Date_reply,Rating,Status,Experience,Date_experience,Response_time
0,Younited Credit,Alexis,2,"[[[], []], [FR]]",Very good service,2023-10-23,"Bonjour,Nous sommes enchantés de vous lire et ...",2023-10-24,5,Verified,Very good service,2023-10-18,1
1,Younited Credit,Exo Saru,3,"[[[], []], [FR]]",Super simple,2023-08-24,"Bonjour,Nous vous remercions d’avoir pris le t...",2023-08-25,5,Verified,"Super simpleSuper simple, très rapide, interfa...",2023-08-17,1
2,Younited Credit,Getifra,3,"[[[], []], [FR]]",Younited Credit a démontré son…,2023-08-05,"Bonjour,Nous vous remercions d’avoir pris le t...",2023-08-08,5,Verified,Younited Credit a démontré son…Younited Credit...,2023-07-27,3
3,Younited Credit,Duval,9,"[[[], []], [FR]]",Rapide efficace simple un pur bonheur…,2022-11-28,"Bonjour,Merci d’avoir pris le temps de nous fa...",2022-11-30,5,Verified,Rapide efficace simple un pur bonheur…Rapide e...,2022-11-28,2
4,Younited Credit,Soso,2,"[[[], []], [FR]]",Younited une procédure simple,2023-02-25,"Bonjour,Nous vous remercions pour ce commentai...",2023-02-27,5,Verified,Younited une procédure simpleYounited permet u...,2023-02-19,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2438,Anytime,AMIR TAOUBAS,1,"[[[], []], [DE]]",Service parfait,2023-03-03,"Bonjour 😁, Merci pour votre super avis ! A trè...",2023-03-06,5,Invited,Service parfait,2023-03-02,3
2439,Anytime,HOCINE TABERKOKT,1,"[[[], []], [FR]]",Excellente communication,2022-05-16,"Bonjour 🫶, Merci beaucoup pour l'avis positif ...",2022-05-17,5,Invited,Excellente communication,2022-05-16,1
2440,Anytime,TOTO JOSUE WENCESLAS KOUAME,1,"[[[], []], [FR]]",Excellent service client,2023-03-08,"Bonjour Toto Josue, Merci de partager votre av...",2023-03-09,5,Invited,Excellent service client,2023-01-31,1
2441,Anytime,SHOLA SANNI,1,"[[[], []], [FR]]",Nice job and keep it up,2022-04-11,Bonjour 😁Merci beaucoup pour cet avis positif....,2022-04-13,5,Invited,Nice job and keep it up,2022-04-11,2


In [38]:
new_df.sort_values(by = "Response_time", ascending=False)

,Company,Customer,Number_review,Language,Title,Date_review,Reply,Date_reply,Rating,Status,Experience,Date_experience,Response_time
1329,Orange Bank,Barbotv,3,"[[[], []], [FR]]",Simple,2022-05-25,"Bonjour,Parfait alors !L’objectif principale e...",2022-06-03,5,Invited,"SimpleSimple, efficace, peu de bug. Correspond...",2022-05-25,9
252,Younited Credit,isabelle,10,"[[[], []], [FR]]",Simple rapide et efficace.,2022-01-22,"Bonjour Madame, Nous vous remercions pour ce ...",2022-01-31,5,Verified,Simple rapide et efficace.,2022-01-22,9
1145,Orange Bank,Fabrice,1,"[[[], []], [RE]]",Service parfait,2022-09-13,"Bonjour Fabrice,Nous nous efforçons d’offrir à...",2022-09-22,5,Invited,Service parfait,2022-09-13,9
1116,Orange Bank,Bruno Thore,2,"[[[], []], [FR]]",Très Bon Relationnel .,2022-09-13,"Bonjour Bruno,Merci de souligner la qualité de...",2022-09-22,5,Invited,Très Bon Relationnel .,2022-09-13,9
1072,Orange Bank,Benjamin,1,"[[[], []], [FR]]",Simple et efficace,2023-01-09,"Bonjour Benjamin, Merci pour ce joli retour ét...",2023-01-18,5,Invited,"Simple et efficaceSimple et efficace, parfait !!",2023-01-08,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
935,Cofidis,PRANGER,6,"[[[], []], [FR]]",Rapide,2021-04-26,Bonjour Pranger et merci pour vos étoiles. Vot...,2021-04-26,5,Invited,"RapideRapide, simple et efficace !",2021-04-26,0
934,Cofidis,MAJDA,1,"[[[], []], [FR]]",Parfait,2021-04-26,Bonjour Majda et merci pour ces étoiles. Je n'...,2021-04-26,5,Invited,"Parfait,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",2021-04-26,0
933,Cofidis,Boubou,1,"[[[], []], [FR]]",Super,2021-04-27,"Bonsoir, merci pour votre commentaire et votre...",2021-04-27,5,Invited,SuperSuper. Merci,2021-04-27,0
932,Cofidis,LL,3,"[[[], []], [FR]]",Interlocuteurs professionnels et a l…,2021-05-11,"Bonjour LL ! Votre satisfaction, c'est notre m...",2021-05-11,5,Invited,Interlocuteurs professionnels et a l…Interlocu...,2021-05-05,0


In [39]:
#new_df["Date_reply"] = new_df["Date_reply"].fillna(new_df["Date_review"])
new_df.isnull().sum()
new_df["Response_time"] = new_df["Response_time"].astype('int64')

# Calculer le temps de réponse moyen pour chaque entreprise
average_response_time_by_company = new_df.groupby('Company')['Response_time'].mean()

# Afficher le résultat
average_response_time_by_company

Company
Anytime             1.897810
BoursoBank          0.000000
Cofidis            15.192913
Floabank            0.622222
Orange Bank        26.101036
Younited Credit    14.384615
Name: Response_time, dtype: float64

In [40]:
new_df.to_csv("../data/processed/avis_clients.csv")

In [41]:
new_df.isna().sum()

Company            0
Customer           0
Number_review      0
Language           0
Title              0
Date_review        0
Reply              0
Date_reply         0
Rating             0
Status             0
Experience         0
Date_experience    0
Response_time      0
dtype: int64

In [1]:
import pandas as pd
df_test = pd.read_csv("../data/processed/reviews.csv", )
#df_test = df_test[["Company", "Customer", "Number_review", "Language", "Title", "Date_review", "Reply", "Date_reply", "Rating", "Status", "Experience", "Date_experience", "Response_time"]]
df_test.sort_values(by="Response_time", ascending=True)

,Company,Customer,Number_review,Language,Title,Date_review,Reply,Date_reply,Rating,Status,Experience,Date_experience,Response_time
714,Younited Credit,Christian René Yves PROVOST,1,FR,excellent,2013-10-25,No Reply,2013-10-25,4,Invited,excellentexcellent excellent excellent,2013-10-25,14
713,Younited Credit,Frederic,1,FR,super!!!,2013-12-03,No Reply,2013-12-03,5,Invited,super!!!très bon placement pour la diversifica...,2013-12-03,53
712,Younited Credit,Pascal,1,FR,SANS SOUCIS,2014-11-20,No Reply,2014-11-20,5,Verified,SANS SOUCISSERVICE IMPECCABLE\rCLAIRE\rNET,2014-11-20,405
711,Younited Credit,CHANTAL LAGRANGE,1,FR,TRES BONNE OPINION,2015-03-19,No Reply,2015-03-19,5,Verified,TRES BONNE OPINIONA RECOMMENDER,2015-03-19,524
710,Younited Credit,alex,1,FR,Excellent,2015-07-09,No Reply,2015-07-09,5,Verified,ExcellentExcellent !!!,2015-07-09,636
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1839,BoursoBank,Gilles,6,FR,Simple et rapide,2023-11-21,No Reply,2023-11-21,5,Invited,Simple et rapide,2023-11-21,3693
715,Cofidis,P.r,1,FR,Un credit parfait,2023-11-20,"Bonjour Priscilla, Merci, ça fait vraiment pla...",2023-11-21,5,Invited,Un credit parfaitUne reponse de principe immed...,2023-11-01,3693
1845,BoursoBank,Thierry MARIET;,3,FR,Nickel,2023-11-24,No Reply,2023-11-24,5,Invited,"Nickelrien a reprocher, double authentificatio...",2023-11-24,3696
1862,BoursoBank,Anaïs S,1,FR,Super application,2023-11-25,No Reply,2023-11-25,5,Invited,Super application,2023-11-23,3697


In [2]:
df_test.isna().sum()

Company            0
Customer           0
Number_review      0
Language           0
Title              0
Date_review        0
Reply              0
Date_reply         0
Rating             0
Status             0
Experience         0
Date_experience    0
Response_time      0
dtype: int64

In [3]:
df_test.isnull().sum()
df_test["Response_time"] = df_test["Response_time"].astype('int64')

# Calculer le temps de réponse moyen pour chaque entreprise
average_response_time_by_company_test = df_test.groupby('Company')['Response_time'].mean()

# Afficher le résultat
average_response_time_by_company_test

Company
Anytime            2983.636719
BoursoBank         3269.240909
Cofidis            3028.350394
Floabank           3056.266667
Orange Bank        3188.831606
Younited Credit    2460.071329
Name: Response_time, dtype: float64

In [2]:
#df_test.isna().sum()
df_test.groupby('Status').count()

,Company,Customer,Number_review,Language,Title,Date_review,Reply,Date_reply,Rating,Experience,Date_experience,Response_time
Status,,,,,,,,,,,,
Invited,1509,1509,1509,1509,1509,1509,1509,1509,1509,1509,1509,1509
Merged,455,455,455,455,455,455,455,455,455,455,455,455
Redirected,18,18,18,18,18,18,18,18,18,18,18,18
Verified,545,545,545,545,545,545,545,545,545,545,545,545
